![Test](screenshot_2634.png "Test")

In [1]:
from wbsurfer.images import Cifti, Gifti
surface_left = (
    "/home/vanandrew/Data/sub-TBIPFM01/NativeVol/fsaverage_LR32k/sub-TBIPFM01.L.very_inflated.32k_fs_LR.surf.gii"
)
parcellation_file = "/home/vanandrew/Data/rawassn_minsize10_regularized_recolored.dscalar.nii"
cifti = Cifti(parcellation_file)
gifti_left = Gifti(surface_left)

In [2]:
from wbsurfer.isomorph import (
    color_lut,
    get_parcellation,
    networks,
    visualize_graph,
)
import numpy as np
from vedo import Text2D
from typing import Dict
def hover_parcel(event: Dict) -> None:
    """Handler for vedo plot mouse interaction

    Parameters
    ----------
    event : Dict
        plot event
    """
    mesh = event.actor
    if not mesh:
        return
    pt = event.picked3d
    # get index of closest point on mesh
    ptid = mesh.closestPoint(pt, returnPointId=True)
    # get the parcel this point belongs to
    for idx, p in enumerate(parcels):
        if ptid in p:
            network = networks[parcellation_nodes[p]["network"]]
            msg.text(f"Network: {network}, Parcel: {idx}, Vertex: {ptid}")
            return
    msg.text(f"Vertex: {ptid}")

In [3]:
# get cifti data
l_vrtable, _, _, r_vrtable, r_start, r_end = cifti.get_vertex_row_table()
row_data = cifti.row_data
# remap 1.5 and 2.5 to no network
row_data[row_data == 1.5] = 0
row_data[row_data == 2.5] = 0
row_data = row_data.astype("u1")[0][:r_end]
# set invalid network assignments
row_data[row_data >= 18] = 0
vrtable = l_vrtable
network_data = row_data[:r_start]
# set left/right surface, network assignment colors
gifti_left.mesh.visual.vertex_colors[l_vrtable] = color_lut[row_data[:r_start]]

In [4]:
# get parcellation representation of the data
parcellation_graph, parcellation_nodes = get_parcellation(gifti_left, vrtable, network_data, threshold=20)
# create a new map that keys by parcellation index
parcellation_map = {int(v["label"]): v for v in parcellation_nodes.values()}
parcels = parcellation_nodes.keys()

In [5]:
# # render mesh
# msg = Text2D(pos='bottom-left', font="VictorMono", c="white")
# gifti_left.render2([("MouseMove", hover_parcel)], [msg])
# visualize graph
visualize_graph(parcellation_graph).show("visual.html")

![](graph.jpg)

![](subgraphs.jpg)

![](full_graph.png)

![](mirror_graph.png)

![](center_graph.png)

In [8]:
# simulate cifti data
c_0 = [5867, 5826, 5866, 5825, 5906]
c_1 = [5827, 5868, 5785, 5786, 5828, 5869]
c_2 = [5829, 5870, 5787, 5744, 5745, 5787, 5829, 5870, 5788, 5830, 5871]
c_3 = [5905, 5865, 5943, 5904, 5864, 5942, 5903]
r_0 = [5743, 5742, 5784, 5741, 5698, 5699]
l_0 = [5907, 5908, 5909, 5947, 5946]
r_1 = [5700, 5701, 5657]
l_1 = [5871, 5910, 5948, 5949, 5911, 5872]
r_2 = [5783, 5740, 5782, 5824, 5696, 5739]
l_2 = [5945, 5983, 5982, 5944, 5981, 5943]
r_3 = [5656, 5610, 5611, 5563, 5564, 5565, 5612]
l_3 = [5950, 5987, 5988]
r_4 = [5697, 5652, 5653, 5607, 5606, 5608, 5654]
l_4 = [5984, 6020, 6021, 6056]
r_5 = [5902, 5862, 5901]
l_5 = [5979, 6015]
fake_network_data = np.zeros(gifti_left.mesh.visual.vertex_colors.shape[0], dtype="u1")
fake_network_data[c_0] = 1
fake_network_data[c_1] = 2
fake_network_data[c_2] = 3
fake_network_data[c_3] = 7
fake_network_data[r_0] = 4
fake_network_data[l_0] = 4
fake_network_data[r_1] = 5
fake_network_data[l_1] = 5
fake_network_data[r_2] = 6
fake_network_data[l_2] = 6
fake_network_data[r_3] = 9
fake_network_data[l_3] = 9
fake_network_data[r_4] = 10
fake_network_data[l_4] = 10
fake_network_data[r_5] = 11
fake_network_data[l_5] = 11
# add network colors to mesh
gifti_left.mesh.visual.vertex_colors = color_lut[fake_network_data]
# create fake data for mesh
fake_vrtable = np.arange(gifti_left.mesh.visual.vertex_colors.shape[0])
vrtable = fake_vrtable
network_data = fake_network_data
parcellation_graph, parcellation_nodes = get_parcellation(gifti_left, vrtable, network_data, threshold=0)
parcels = parcellation_nodes.keys()
parcellation_map = {int(v["label"]): v for v in parcellation_nodes.values()}

In [9]:
# render mesh
msg = Text2D(pos='bottom-left', font="VictorMono", c="white")
gifti_left.render2([("MouseMove", hover_parcel)], [msg])
# visualize the graph
visualize_graph(parcellation_graph).show("visual.html")